In [2]:
import importlib
import numpy as np
import pandas as pd
import utils

In [3]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/stacked_lstm_statefull/utils.py'>

In [4]:
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Bidirectional
from keras.preprocessing.sequence import pad_sequences
# from keras.optimizers import Adam

Using TensorFlow backend.


In [5]:
word2ind, ind2word = utils.load_index_word_map()

In [6]:
vocab_size = len(word2ind)

In [9]:
dataset = pd.read_csv('data/train_1002.csv')

In [10]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,36,70,67,1,61,64,71,54,1,58,...,68,65,58,61,61,68,1,68,64,62
1,36,57,7,1,72,54,1,50,67,54,...,53,1,50,1,57,54,50,67,69,1
2,40,70,68,65,58,52,58,64,63,68,...,54,53,1,1,0,46,64,70,1,68
3,5,22,63,56,54,61,1,51,50,51,...,68,69,50,67,69,68,1,62,54,1
4,44,57,54,63,1,74,64,70,4,67,...,57,1,68,57,50,52,60,1,1,0


In [11]:
dataset.shape

(49610, 101)

In [12]:
emb_dim = 100
input_length = 100
batch_size = 55

In [13]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, batch_input_shape=(batch_size, input_length)))
model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Dropout(.2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=.2, recurrent_dropout=.2, stateful=True)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(100, dropout=.2, recurrent_dropout=.2, stateful=True)))
model.add(BatchNormalization())
model.add(Dropout(.2))
model.add(Dense(vocab_size, activation='softmax'))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (55, 100, 100)            7600      
_________________________________________________________________
conv1d_1 (Conv1D)            (55, 100, 256)            102656    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (55, 50, 256)             0         
_________________________________________________________________
dropout_1 (Dropout)          (55, 50, 256)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (55, 50, 32)              32800     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (55, 25, 32)              0         
_________________________________________________________________
bidirectional_1 (Bidirection (55, 25, 200)             106400    
__________

In [15]:
model.set_weights(load_model('model.h5').get_weights())

In [16]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
for _ in range(50):
    model.reset_states()
    model.fit(dataset.iloc[:, :100].values, dataset.iloc[:, [100]].values, epochs=1, batch_size=batch_size)

Epoch 1/1
49610/49610 [==============================] - 56s - loss: 1.7724 - acc: 0.5066    
Epoch 1/1
49610/49610 [==============================] - 53s - loss: 1.6192 - acc: 0.5350    
Epoch 1/1
49610/49610 [==============================] - 53s - loss: 1.5595 - acc: 0.5485    
Epoch 1/1
49610/49610 [==============================] - 53s - loss: 1.5126 - acc: 0.5582    
Epoch 1/1
49610/49610 [==============================] - 53s - loss: 1.4834 - acc: 0.5670    
Epoch 1/1
49610/49610 [==============================] - 53s - loss: 1.4512 - acc: 0.5741    
Epoch 1/1
49610/49610 [==============================] - 52s - loss: 1.4309 - acc: 0.5763    
Epoch 1/1
49610/49610 [==============================] - 53s - loss: 1.4031 - acc: 0.5851    
Epoch 1/1
49610/49610 [==============================] - 52s - loss: 1.3868 - acc: 0.5906    
Epoch 1/1
49610/49610 [==============================] - 52s - loss: 1.3596 - acc: 0.5979    
Epoch 1/1
49610/49610 [==============================] - 53s

In [18]:
# save model
model.save('model.h5')

In [19]:
pred_model = Sequential()
pred_model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, batch_input_shape=(1, input_length)))
pred_model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=.2, recurrent_dropout=.2, stateful=True)))
pred_model.add(BatchNormalization())
pred_model.add(Bidirectional(LSTM(100, dropout=.2, recurrent_dropout=.2, stateful=True)))
pred_model.add(BatchNormalization())
pred_model.add(Dense(vocab_size, activation='softmax'))

In [20]:
pred_model.set_weights(model.get_weights())

In [21]:
def implement(seed_text, maxlen=100, must_stop=2000):
    cleaned = list(seed_text)
    padded_input_tokens = utils.tokenise(word2ind, cleaned)
    res_tokens = [token for token in padded_input_tokens]
    while must_stop > 0:
        padded_input_tokens = pad_sequences([padded_input_tokens], maxlen=maxlen)
        probs = pred_model.predict(padded_input_tokens, batch_size=1)[0]
        first_n = np.argsort(probs)[-3:]
        probs = probs[first_n] / np.sum(probs[first_n])
        predicted = np.random.choice(first_n, p=probs)
        padded_input_tokens = padded_input_tokens[0].tolist()
        padded_input_tokens.append(predicted)
        res_tokens.append(predicted)
        must_stop -= 1
    detokenised = utils.detokenise(ind2word, res_tokens)
    return ''.join(detokenised)

In [22]:
res = implement('We are friends, good friends', 100, 200)

In [23]:
print(res)

We are friends, good friends,, some me and I mean a man, mach my,, can't pack to pack  
I'm we loving and they long all the little the logs that you can  
Happine in shadows got that there withing her shill  
Angel baby, yeah  

